In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

In [2]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')

In [3]:
train.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [5]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)

In [6]:
#train을 target과 feature로 나눠줍니다.
train_ = train.dropna(axis=0)
train_x = train_.drop(['nerdiness'],axis=1)
train_y = train_['nerdiness']

In [7]:
train_scaled = StandardScaler().fit_transform(train_.iloc[:, :-1])
pca = PCA(n_components=30)

In [8]:
pca.fit(train_scaled)
train_pca = pca.transform(train_scaled)

In [9]:
pca_columns=['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',]
trainDF_pca = pd.DataFrame(train_pca, columns=pca_columns)
trainDF_pca['nerdiness']=train_y

In [21]:
pca_X = trainDF_pca[['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',]]

0        1
1        1
2        1
3        1
4        0
        ..
14995    0
14996    1
14997    1
14998    0
14999    1
Name: nerdiness, Length: 12796, dtype: int64

In [25]:
rcf = RandomForestClassifier(random_state=156)
rcf = rcf.fit(train_pca[:, :-1],train_y)
scores_pca = cross_val_score(rcf, pca_X, train_y, scoring='accuracy', cv=2 )
rcf_pred = rcf.predict(test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
submission = pd.read_csv('competition_data/sample_submission.csv')
submission

In [ ]:
submission["nerdiness"] = rcf_pred

In [ ]:
submission

In [ ]:
submission.to_csv("baseline.csv", index = False)